In [128]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

## Labelling of train and test data

In [131]:
import os
df_train=pd.read_excel('oneBotTrainingData.xlsx',sheet_name='Train')#oneBotTrainingData.xlsx
df_test=pd.read_excel('GoldenCopyDataFile.xlsx')#GoldenCopyDataFile.xlsx

## Cleaning for train data

In [132]:
import re
def clean_text(text):
    text=text.lower()
#     text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

df_train['utterance']=df_train['utterance'].apply(lambda x: clean_text(x))
df_train['utterance']=df_train['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_train['utterance']=df_train['utterance'].str.replace(".", "")

# selecting on > 2 len sentence utterances
df_trainutt=df_train["utterance"].values
df_trainutt
s = pd.Series(df_trainutt)
df_train["utt"]=s.str.split().to_frame()
type(s.str.split().to_frame())
df_train['Length'] = df_train['utt'].str.len()
df_train
#last step
df_train.query('Length > 2', inplace = True) 
df_train=df_train[["utterance","intent"]]
df_train

utterance  \
0                am i required to do mandatory learning    
1      are there any mandatory e learning courses tha...   
2      are there any mandatory elearnings i need to c...   
3      can you provide the list of mandatory training...   
4      can you tell me the mandatory trainings i need...   
...                                                  ...   
18287              how do i delegate timesheet approvals   
18288                delegate timesheet approval in cwms   
18289       can i delegate acpc staff timesheet approval   
18290         delegate contract staff timesheet approval   
18291                     misc fee approval notification   

                                                  intent  
0      training.who_is_required_to_complete_the_manda...  
1      training.who_is_required_to_complete_the_manda...  
2      training.who_is_required_to_complete_the_manda...  
3      training.who_is_required_to_complete_the_manda...  
4      training.who_is_required_to_complete_the_manda...  
...                                                  ...  
18287              cwms.approvals_and_delegation_in_cwms  
18288              cwms.approvals_and_delegation_in_cwms  
18289              cwms.approvals_and_delegation_in_cwms  
18290              cwms.approvals_and_delegation_in_cwms  
18291       cwms.timesheet_and_invoicing_matters_in_cwms  

[17005 rows x 2 columns]

## Cleaning for test data

In [133]:
import re
def clean_text(text):
    text=text.lower()
#     text=re.sub('\w*\d\w*','', text)
    #text=re.sub('\n',' ',text)
    #text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text
df_test['utterance']=df_test['utterance'].astype(str)
df_test['utterance']=df_test['utterance'].apply(lambda x: clean_text(x))

df_test['utterance']

df_test['utterance']=df_test['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_test['utterance']=df_test['utterance'].str.replace(".", "")

df_test=df_test[["utterance","intent"]]
df_test

utterance  \
0                                     1bank id   
1                3 tenets of reward philosophy   
2                                      3 tenet   
3                                     3 tenets   
4                            about be my guest   
...                                        ...   
3495                why need guided min salary   
3496  how often is our minimum salary reviewed   
3497      will bank review our guided min wage   
3498           purpose of setting minimum wage   
3499            what affects guided min salary   

                                                 intent  
0     onboarding.when_will_the_new_hire_1bank_id_be_...  
1           compensation.what_is_dbs_rewards_philosophy  
2           compensation.what_is_dbs_rewards_philosophy  
3           compensation.what_is_dbs_rewards_philosophy  
4                         talent_management.be_my_guest  
...                                                 ...  
3495  compensation.how_to_set_and_why_we_need_guided...  
3496  compensation.how_to_set_and_why_we_need_guided...  
3497  compensation.how_to_set_and_why_we_need_guided...  
3498  compensation.how_to_set_and_why_we_need_guided...  
3499  compensation.how_to_set_and_why_we_need_guided...  

[3500 rows x 2 columns]

In [134]:
df_test['utterance'][df_test['utterance']=='']='NONE'

## Labelling

In [135]:
#df=pd.concat([df_train[['utterance','intent']],df_test[['utterance','intent']]])
le=preprocessing.LabelEncoder()
le.fit(df_train['intent'])

df_train['intent1']=le.transform(df_train['intent'])
mapping=df_train[['intent','intent1']].drop_duplicates().rename(columns={'intent1':'mapping'})

df_test=pd.merge(df_test,mapping,on='intent',how='left').rename(columns={'mapping':'intent1'})

df_test['intent1']=df_test['intent1'].fillna(0).astype(int)

In [136]:
df_train

utterance  \
0                am i required to do mandatory learning    
1      are there any mandatory e learning courses tha...   
2      are there any mandatory elearnings i need to c...   
3      can you provide the list of mandatory training...   
4      can you tell me the mandatory trainings i need...   
...                                                  ...   
18287              how do i delegate timesheet approvals   
18288                delegate timesheet approval in cwms   
18289       can i delegate acpc staff timesheet approval   
18290         delegate contract staff timesheet approval   
18291                     misc fee approval notification   

                                                  intent  intent1  
0      training.who_is_required_to_complete_the_manda...      487  
1      training.who_is_required_to_complete_the_manda...      487  
2      training.who_is_required_to_complete_the_manda...      487  
3      training.who_is_required_to_complete_the_manda...      487  
4      training.who_is_required_to_complete_the_manda...      487  
...                                                  ...      ...  
18287              cwms.approvals_and_delegation_in_cwms       18  
18288              cwms.approvals_and_delegation_in_cwms       18  
18289              cwms.approvals_and_delegation_in_cwms       18  
18290              cwms.approvals_and_delegation_in_cwms       18  
18291       cwms.timesheet_and_invoicing_matters_in_cwms       28  

[17005 rows x 3 columns]

In [137]:
mapping

intent  mapping
0      training.who_is_required_to_complete_the_manda...      487
90          training.who_is_my_business_learning_partner      486
111    training.where_do_i_do_complete_my_course_eval...      485
131      training.where_can_i_view_my_completed_learning      484
212    training.where_can_i_locate_my_e-learnings_in_...      483
...                                                  ...      ...
18105      training.how_do_i_check_my_skillsflex_balance      449
18139                        shares.special_shares_award      389
18170         general_benefits.service_recognition_award       50
18209                 compensation.guided_minimum_salary        3
18240  compensation.how_to_set_and_why_we_need_guided...        9

[488 rows x 2 columns]

In [145]:
df_test

utterance  \
0                                     1bank id   
1                3 tenets of reward philosophy   
2                                      3 tenet   
3                                     3 tenets   
4                            about be my guest   
...                                        ...   
3495                why need guided min salary   
3496  how often is our minimum salary reviewed   
3497      will bank review our guided min wage   
3498           purpose of setting minimum wage   
3499            what affects guided min salary   

                                                 intent  intent1  
0     onboarding.when_will_the_new_hire_1bank_id_be_...      211  
1           compensation.what_is_dbs_rewards_philosophy       14  
2           compensation.what_is_dbs_rewards_philosophy       14  
3           compensation.what_is_dbs_rewards_philosophy       14  
4                         talent_management.be_my_guest      423  
...                                                 ...      ...  
3495  compensation.how_to_set_and_why_we_need_guided...        9  
3496  compensation.how_to_set_and_why_we_need_guided...        9  
3497  compensation.how_to_set_and_why_we_need_guided...        9  
3498  compensation.how_to_set_and_why_we_need_guided...        9  
3499  compensation.how_to_set_and_why_we_need_guided...        9  

[3500 rows x 3 columns]

## Save the files

In [146]:
df_train.to_excel('train_onebot.xlsx',index=None)

In [147]:
writer=pd.ExcelWriter('gold_test.xlsx',engine='xlsxwriter')

In [148]:
df_test.to_excel(writer,sheet_name='Test',index=None)
mapping.to_excel(writer,sheet_name='Mapping',index=None)

In [149]:
writer.save()